<a href="https://colab.research.google.com/github/thursy/GCP-BQML/blob/master/BQML_TaxiFarePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BigQuery Machine Learning - Predict Taxi Fare with a BigQuery ML Forecasting Model


Predict Taxi Fare with a BigQuery ML Forecasting Model
After making a project and enable the API, here are the steps to do Machine Learning using BigQuery

* Step one: Use BigQuery to find public datasets.
* Step two: Query and explore the public taxi cab dataset.
* Step three: Create a training and evaluation dataset to be used * for batch prediction.
* Step four: Create a forecasting (linear regression) model in BQML.
* Step five: Evaluate the performance of your machine learning model.
* Step six: Predict taxi fare amount.
* Step seven: Improving the model with Feature Engineering


###In case using Google Colab, user Authentication is needed by running the code below

In [0]:
from google.colab import auth
auth.authenticate_user()

##Query and explore the public taxi cab dataset.


In [5]:
%%bigquery --project tutorial-medium-2020
SELECT
  TIMESTAMP_TRUNC(pickup_datetime,
    MONTH) month,
  COUNT(*) trips
FROM
  `bigquery-public-data.new_york.tlc_yellow_trips_2015`
GROUP BY
  1
ORDER BY
  1

,month,trips
0,2015-01-01 00:00:00+00:00,12748986
1,2015-02-01 00:00:00+00:00,12450521
2,2015-03-01 00:00:00+00:00,13351609
3,2015-04-01 00:00:00+00:00,13071789
4,2015-05-01 00:00:00+00:00,13158262
5,2015-06-01 00:00:00+00:00,12324935
6,2015-07-01 00:00:00+00:00,11562783
7,2015-08-01 00:00:00+00:00,11130304
8,2015-09-01 00:00:00+00:00,11225063
9,2015-10-01 00:00:00+00:00,12315488


In [6]:
%%bigquery --project tutorial-medium-2020
SELECT
  EXTRACT(HOUR
  FROM
    pickup_datetime) hour,
  ROUND(AVG(trip_distance / TIMESTAMP_DIFF(dropoff_datetime,
        pickup_datetime,
        SECOND))*3600, 1) speed
FROM
  `bigquery-public-data.new_york.tlc_yellow_trips_2015`
WHERE
  trip_distance > 0
  AND fare_amount/trip_distance BETWEEN 2
  AND 10
  AND dropoff_datetime > pickup_datetime
GROUP BY
  1
ORDER BY
  1


,hour,speed
0,0,15.8
1,1,16.3
2,2,16.8
3,3,17.5
4,4,20.0
5,5,21.6
6,6,17.6
7,7,13.7
8,8,11.6
9,9,11.4


##Create a training and evaluation dataset to be used for batch prediction

In [7]:
%%bigquery --project tutorial-medium-2020
WITH params AS (
    SELECT
    1 AS TRAIN,
    2 AS EVAL
    ),
  daynames AS
    (SELECT ['Sun', 'Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat'] AS daysofweek),
  taxitrips AS (
  SELECT
    (tolls_amount + fare_amount) AS total_fare,
    daysofweek[ORDINAL(EXTRACT(DAYOFWEEK FROM pickup_datetime))] AS dayofweek,
    EXTRACT(HOUR FROM pickup_datetime) AS hourofday,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat,
    passenger_count AS passengers
  FROM
    `nyc-tlc.yellow.trips`, daynames, params
  WHERE
    trip_distance > 0 AND fare_amount > 0
    AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))),1000) = params.TRAIN
  )

  SELECT *
  FROM taxitrips


,total_fare,dayofweek,hourofday,pickuplon,pickuplat,dropofflon,dropofflat,passengers
0,46.33,Mon,0,-73.865225,40.770845,-74.009762,40.705505,1
1,18.10,Sat,0,-73.943430,40.815628,-73.904790,40.880418,2
2,18.90,Fri,0,-73.993182,40.742235,-73.948297,40.801398,1
3,11.80,Tues,0,-73.995963,40.726475,-73.966875,40.688341,1
4,15.70,Sat,0,-74.015413,40.706540,-73.991632,40.726781,1
...,...,...,...,...,...,...,...,...
1120072,5.70,Tues,23,-74.004260,40.720912,-74.004272,40.733405,5
1120073,7.70,Tues,23,0.000000,0.000000,0.000000,0.000000,2
1120074,7.70,Wed,23,-73.972533,40.793249,-73.946604,40.789008,1
1120075,7.70,Fri,23,-73.974750,40.759292,-73.953147,40.767632,1


##Create a forecasting (linear regression) model in BQML


In [9]:
%%bigquery --project tutorial-medium-2020

CREATE or REPLACE MODEL taxi.taxifare_model
OPTIONS
  (model_type='linear_reg', labels=['total_fare']) AS

WITH params AS (
    SELECT
    1 AS TRAIN,
    2 AS EVAL
    ),

  daynames AS
    (SELECT ['Sun', 'Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat'] AS daysofweek),

  taxitrips AS (
  SELECT
    (tolls_amount + fare_amount) AS total_fare,
    daysofweek[ORDINAL(EXTRACT(DAYOFWEEK FROM pickup_datetime))] AS dayofweek,
  EXTRACT(HOUR FROM pickup_datetime) AS hourofday,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat,
    passenger_count AS passengers
  FROM
    `nyc-tlc.yellow.trips`, daynames, params
  WHERE
    trip_distance > 0 AND fare_amount > 0
    AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))),1000) = params.TRAIN
  )

  SELECT *
  FROM taxitrips



""


##Evaluate the performance of your machine learning model

In [10]:
%%bigquery --project tutorial-medium-2020
SELECT
  SQRT(mean_squared_error) AS rmse
FROM
  ML.EVALUATE(MODEL taxi.taxifare_model,
  (

  WITH params AS (
    SELECT
    1 AS TRAIN,
    2 AS EVAL
    ),

  daynames AS
    (SELECT ['Sun', 'Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat'] AS daysofweek),

  taxitrips AS (
  SELECT
    (tolls_amount + fare_amount) AS total_fare,
    daysofweek[ORDINAL(EXTRACT(DAYOFWEEK FROM pickup_datetime))] AS dayofweek,
  EXTRACT(HOUR FROM pickup_datetime) AS hourofday,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat,
    passenger_count AS passengers
  FROM
    `nyc-tlc.yellow.trips`, daynames, params
  WHERE
    trip_distance > 0 AND fare_amount > 0
    AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))),1000) = params.EVAL
  )

  SELECT *
  FROM taxitrips))


,rmse
0,9.476637


After evaluating your model you get a RMSE of 9.47. Since we took the Root of the Mean Squared Error (RMSE) the 9.47 error can be evaluated in the same units as the total_fare so it's +-$9.47.


##Predict taxi fare amount


In [11]:
%%bigquery --project tutorial-medium-2020
SELECT
*
FROM
  ml.PREDICT(MODEL `taxi.taxifare_model`,
   (

 WITH params AS (
    SELECT
    1 AS TRAIN,
    2 AS EVAL
    ),

  daynames AS
    (SELECT ['Sun', 'Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat'] AS daysofweek),

  taxitrips AS (
  SELECT
    (tolls_amount + fare_amount) AS total_fare,
    daysofweek[ORDINAL(EXTRACT(DAYOFWEEK FROM pickup_datetime))] AS dayofweek,
  EXTRACT(HOUR FROM pickup_datetime) AS hourofday,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat,
    passenger_count AS passengers
  FROM
    `nyc-tlc.yellow.trips`, daynames, params
  WHERE
    trip_distance > 0 AND fare_amount > 0
    AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))),1000) = params.EVAL
  )

  SELECT *
  FROM taxitrips

));


,predicted_total_fare,total_fare,dayofweek,hourofday,pickuplon,pickuplat,dropofflon,dropofflat,passengers
0,11.688069,4.0,Sun,0,-73.995217,40.744731,-74.001528,40.739350,1
1,11.337108,4.5,Sat,0,-73.984926,40.727959,-73.985559,40.721855,2
2,11.816619,6.5,Mon,0,-73.985145,40.747816,-73.981212,40.762677,3
3,11.396162,6.5,Tues,0,-73.988154,40.721315,-74.000781,40.729677,1
4,11.852903,6.5,Tues,0,-74.004803,40.730222,-73.993570,40.713775,5
...,...,...,...,...,...,...,...,...,...
1111101,11.111809,5.7,Sun,23,-73.999952,40.743293,-73.998568,40.731720,1
1111102,11.071862,7.7,Thurs,23,-73.988725,40.722317,-73.993330,40.741452,1
1111103,10.646699,7.7,Sat,23,-73.967362,40.756584,-73.988532,40.768271,1
1111104,11.122983,8.9,Fri,23,-73.989663,40.739461,-74.010832,40.722274,1


##Improving the model with Feature Engineering
Filtering the training dataset. 
Check on the lowest and the highest fare. Notice that the highest fare is over $50,000.


In [12]:
%%bigquery --project tutorial-medium-2020
SELECT
  COUNT(fare_amount) AS num_fares,
  MIN(fare_amount) AS low_fare,
  MAX(fare_amount) AS high_fare,
  AVG(fare_amount) AS avg_fare,
  STDDEV(fare_amount) AS stddev
FROM
`nyc-tlc.yellow.trips`
# 1,108,779,463 fares


,num_fares,low_fare,high_fare,avg_fare,stddev
0,1108779463,-21474808.0,503325.53,11.105719,650.44458


Filtering the training dataset. 
Limit the data from $$6 to $200. Yet we still have 800 million rides for our model to learn from

In [13]:
%%bigquery --project tutorial-medium-2020
SELECT
  COUNT(fare_amount) AS num_fares,
  MIN(fare_amount) AS low_fare,
  MAX(fare_amount) AS high_fare,
  AVG(fare_amount) AS avg_fare,
  STDDEV(fare_amount) AS stddev
FROM
`nyc-tlc.yellow.trips`
WHERE trip_distance > 0 AND fare_amount BETWEEN 6 and 200
# 843,834,902 fares


,num_fares,low_fare,high_fare,avg_fare,stddev
0,843834902,6.0,200.0,12.992424,9.152008


Filtering the training dataset. 
While you're at, let's limit the distance traveled so you're really focusing on New York City.


In [14]:
%%bigquery --project tutorial-medium-2020
SELECT
  COUNT(fare_amount) AS num_fares,
  MIN(fare_amount) AS low_fare,
  MAX(fare_amount) AS high_fare,
  AVG(fare_amount) AS avg_fare,
  STDDEV(fare_amount) AS stddev
FROM
`nyc-tlc.yellow.trips`
WHERE trip_distance > 0 AND fare_amount BETWEEN 6 and 200
    AND pickup_longitude > -75 #limiting of the distance the taxis travel out
    AND pickup_longitude < -73
    AND dropoff_longitude > -75
    AND dropoff_longitude < -73
    AND pickup_latitude > 40
    AND pickup_latitude < 42
    AND dropoff_latitude > 40
    AND dropoff_latitude < 42
    # 827,365,869 fares



,num_fares,low_fare,high_fare,avg_fare,stddev
0,827365869,6.0,200.0,12.989136,9.139808


Retraining the model

In [16]:
%%bigquery --project tutorial-medium-2020
CREATE OR REPLACE MODEL taxi.taxifare_model_2
OPTIONS
  (model_type='linear_reg', labels=['total_fare']) AS
WITH params AS (
    SELECT
    1 AS TRAIN,
    2 AS EVAL
    ),
  daynames AS
    (SELECT ['Sun', 'Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat'] AS daysofweek),
  taxitrips AS (
  SELECT
    (tolls_amount + fare_amount) AS total_fare,
    daysofweek[ORDINAL(EXTRACT(DAYOFWEEK FROM pickup_datetime))] AS dayofweek,
    EXTRACT(HOUR FROM pickup_datetime) AS hourofday,
    SQRT(POW((pickup_longitude - dropoff_longitude),2) + POW(( pickup_latitude - dropoff_latitude), 2)) as dist, #Euclidean distance between pickup and drop off
    SQRT(POW((pickup_longitude - dropoff_longitude),2)) as longitude, #Euclidean distance between pickup and drop off in longitude
    SQRT(POW((pickup_latitude - dropoff_latitude), 2)) as latitude, #Euclidean distance between pickup and drop off in latitude
    passenger_count AS passengers
  FROM
    `nyc-tlc.yellow.trips`, daynames, params
  WHERE trip_distance > 0 AND fare_amount BETWEEN 6 and 200
    AND pickup_longitude > -75 #limiting of the distance the taxis travel out
    AND pickup_longitude < -73
    AND dropoff_longitude > -75
    AND dropoff_longitude < -73
    AND pickup_latitude > 40
    AND pickup_latitude < 42
    AND dropoff_latitude > 40
    AND dropoff_latitude < 42
    AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))),1000) = params.TRAIN
  )
 SELECT *
  FROM taxitrips



""


Evaluate the new model


In [17]:
%%bigquery --project tutorial-medium-2020
SELECT
  SQRT(mean_squared_error) AS rmse
FROM
  ML.EVALUATE(MODEL taxi.taxifare_model_2,
  (
  WITH params AS (
    SELECT
    1 AS TRAIN,
    2 AS EVAL
    ),
  daynames AS
    (SELECT ['Sun', 'Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat'] AS daysofweek),
  taxitrips AS (
  SELECT
    (tolls_amount + fare_amount) AS total_fare,
    daysofweek[ORDINAL(EXTRACT(DAYOFWEEK FROM pickup_datetime))] AS dayofweek,
    EXTRACT(HOUR FROM pickup_datetime) AS hourofday,
    SQRT(POW((pickup_longitude - dropoff_longitude),2) + POW(( pickup_latitude - dropoff_latitude), 2)) as dist, #Euclidean distance between pickup and drop off
    SQRT(POW((pickup_longitude - dropoff_longitude),2)) as longitude, #Euclidean distance between pickup and drop off in longitude
    SQRT(POW((pickup_latitude - dropoff_latitude), 2)) as latitude, #Euclidean distance between pickup and drop off in latitude
    passenger_count AS passengers
  FROM
    `nyc-tlc.yellow.trips`, daynames, params
  WHERE trip_distance > 0 AND fare_amount BETWEEN 6 and 200
    AND pickup_longitude > -75 #limiting of the distance the taxis travel out
    AND pickup_longitude < -73
    AND dropoff_longitude > -75
    AND dropoff_longitude < -73
    AND pickup_latitude > 40
    AND pickup_latitude < 42
    AND dropoff_latitude > 40
    AND dropoff_latitude < 42
    AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))),1000) = params.EVAL
  )
  SELECT *
  FROM taxitrips))



,rmse
0,5.124653


As you see, you've now gotten the RMSE down to: +-$$5.12 which is significantly better than +-$9.47 for your first model.
Since RSME defines the standard deviation of prediction errors, we see that the retrained linear regression made our model a lot more accurate.
